# Attainment of Scholar subsets for training of gFedNTM

In [10]:
sc.version

NameError: name 'sc' is not defined

In [9]:
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from difflib import SequenceMatcher, get_close_matches
import pandas as pd
from pyspark.sql.functions import col, explode, regexp_replace, length, lit, year, split, arrays_zip

ModuleNotFoundError: No module named 'sparknlp'

In [11]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path

In [12]:
path_scholar_raw = Path("/export/usuarios_ml4ds/lbartolome/scholar/SemanticScholar/20220201/papers.parquet")
path_scholar_lemmas = Path("/export/data_ml4ds/IntelComp/Datasets/lemmatization_output/lemmatization_output_Sep_2022/output_semantic")
path_lemmas_federated_small = Path("/export/usuarios_ml4ds/lbartolome/data/scholar_federated_small")
path_lemmas_federated_tiny = Path("/export/usuarios_ml4ds/lbartolome/data/scholar_federated_tiny")
path_embeddings = Path("/export/data_ml4ds/IntelComp/Datasets/Datasets_embeddings/scholar-embeddings_v2.parquet")
path_output_small = Path("/export/usuarios_ml4ds/lbartolome/gFedNTM/aux/corpus.parquet")
path_output_tiny = Path("/export/usuarios_ml4ds/lbartolome/gFedNTM/aux_small/corpus.parquet")

In [4]:
path_lemmas_federated = path_lemmas_federated_small
path_output = path_output_small

In [13]:
df = dd.read_parquet(path_scholar_raw)
df = df.sample(frac=0.00001, replace=True, random_state=1)
#df2 = df.compute()

In [14]:
df.columns

Index(['id', 'title', 'paperAbstract', 's2Url', 'pdfUrls', 'year', 'sources',
       'doi', 'doiUrl', 'pmid', 'magId', 'fieldsOfStudy', 'journalName',
       'journalPages', 'journalVolume', 'venue'],
      dtype='object')

In [16]:
df2 = df[df.fieldsOfStudy.str.contains('Computer Science')]

In [11]:
df2 = df[df['fieldsOfStudy'].str.contains(r'\"Computer\sScience"', regex=True)]

In [17]:
df2.compute().head()

KeyError: "None of [Float64Index([nan], dtype='float64')] are in the [columns]"

## 1. Get categories in Scholar

In [5]:
S2papers = spark.read.parquet(f"file://{path_scholar_raw}")

#Concatenate text fields from which lemmas are calculated
S2papers = (
    S2papers.withColumn("text",F.concat_ws('. ', "title", "paperAbstract"))
    .drop("title")
    .drop("paperAbstract").select("id", "year","fieldsOfStudy")
)

In [6]:
S2papers

DataFrame[id: string, year: bigint, fieldsOfStudy: array<string>]

In [7]:
print('Number of papers in Scholar:', S2papers.count())

Number of papers in Scholar: 204457855


In [8]:
years = S2papers.select('year').distinct().toPandas()
y = sorted(years.values.tolist())#1961

In [9]:
categories = S2papers.select('fieldsOfStudy').distinct().toPandas()
categories

,fieldsOfStudy
0,"[Economics, Medicine]"
1,"[Medicine, Physics, Materials Science]"
2,"[Biology, Geology, Engineering]"
3,"[Engineering, Psychology, Physics]"
4,"[Art, History, Medicine]"
...,...
10119,"[Chemistry, Mathematics, Biology, Physics]"
10120,"[Art, Philosophy, Computer Science, Political ..."
10121,"[Art, Psychology, Biology]"
10122,"[Chemistry, Business, Economics]"


In [10]:
papers_s2cs = S2papers.filter(F.array_contains("fieldsOfStudy", 'Computer Science'))
count_s2cs = papers_s2cs.count()
print('Number of papers in Computer Science:', count_s2cs)

papers_s2eco = S2papers.filter(F.array_contains("fieldsOfStudy", 'Economics'))
count_s2eco = papers_s2eco.count()
print('Number of papers in Economics:', count_s2eco)

papers_s2socio = S2papers.filter(F.array_contains("fieldsOfStudy", 'Sociology'))
count_s2socio =  papers_s2socio.count()
print('Number of papers in Sociology:', count_s2socio)

papers_s2phi = S2papers.filter(F.array_contains("fieldsOfStudy", 'Environmental Science'))
count_s2phi = papers_s2phi.count()
print('Number of papers in Environmental Science:', count_s2phi)

papers_s2ps = S2papers.filter(F.array_contains("fieldsOfStudy", 'Political Science'))
count_s2ps = papers_s2ps.count()
print('Number of papers in Political Science:', count_s2ps)

Number of papers in Computer Science: 14801864


Number of papers in Economics: 3266129


Number of papers in Sociology: 5039428


Number of papers in Environmental Science: 3231822


Number of papers in Political Science: 7100508


In [11]:
total = 3000

In [12]:
papers_s2cs = S2papers.filter(F.array_contains("fieldsOfStudy", 'Computer Science')).sample(fraction=total/count_s2cs)
print('Number of papers in Computer Science:', papers_s2cs.count())

Number of papers in Computer Science: 3027


In [13]:
papers_s2eco = S2papers.filter(F.array_contains("fieldsOfStudy", 'Economics')).sample(fraction=total/count_s2eco)
print('Number of papers in Economics:', papers_s2eco.count())

Number of papers in Economics: 2998


In [14]:
papers_s2socio = S2papers.filter(F.array_contains("fieldsOfStudy", 'Sociology')).sample(fraction=total/count_s2socio)
print('Number of papers in Sociology:', papers_s2socio.count())

Number of papers in Sociology: 2963


In [15]:
papers_s2phi = S2papers.filter(F.array_contains("fieldsOfStudy", 'Philosophy')).sample(fraction=total/count_s2phi)
print('Number of papers in Philosophy:', papers_s2phi.count())

Number of papers in Environmental Science: 2996


In [16]:
papers_s2ps = S2papers.filter(F.array_contains("fieldsOfStudy", 'Political Science')).sample(fraction=total/count_s2ps)
print('Number of papers in Political Science:', papers_s2ps.count())

Number of papers in Political Science: 2971


## 2. Get lemmas of selected fields with POS [NOUN, VERB, ADJ]


In [17]:
S2lemmas = spark.read.parquet(f"file://{path_scholar_lemmas}")

S2lemmas = (
    S2lemmas.withColumn("lemmas",F.concat_ws(' ', "title_lemmas", "paperAbstract_lemmas"))
    .withColumn("embeddings",F.concat_ws('. ', "title_embeddings", "paperAbstract_embeddings"))
    .withColumn("pos",F.concat_ws(' ', "title_pos", "paperAbstract_pos"))
    .withColumn("lemmas_list", split("lemmas", " "))
    .withColumn("pos_list", split("pos", " "))
    .select("id", "embeddings", "lemmas_list", "pos_list")
)
S2lemmas = S2lemmas.where(length(col("lemmas")) > 5)
S2lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------
 id          | 67a86b9076b2bfcc8fdc3054c4b6215ae4a69129                                                                                 
 embeddings  | 0.047143493. -0.4874212. 0.14710653. -0.34913865. -0.030980468. -0.6288803. 0.30476338. -0.04096819. 0.64699614. -0.5... 
 lemmas_list | [spatial, dealiasing, of, 3-d, seismic, reflection, datum, SUMMARY, one, of, the, main, challenge, of, 3-d, reflectio... 
 pos_list    | [ADJ, NOUN, ADP, NUM, ADJ, NOUN, NOUN, PROPN, NUM, ADP, DET, ADJ, NOUN, ADP, NUM, NOUN, NOUN, AUX, VERB, DET, ADJ, NO... 
-RECORD 1-------------------------------------------------------------------------------------------------------------------------------
 id          | 30e2c1e54a88f22513458f53129e04de590ee515                                                                                 
 embeddings  | 0.0045647793. -0.3976266. 

In [18]:
from pyspark.sql.functions import expr
S2lemmas = (S2lemmas.select("id", "embeddings", expr("""
                    filter(
                    arrays_zip(lemmas_list, pos_list),
                    e -> e.pos_list == 'ADJ' or e.pos_list == 'NOUN' or e.pos_list == 'VERB'
                    ).lemmas_list AS result
                    """))
                   .withColumn("lemmas",F.concat_ws(' ', "result"))
                   .select("id", "lemmas")
                  )
S2lemmas.show(n=4, truncate=120, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------
 id     | 67a86b9076b2bfcc8fdc3054c4b6215ae4a69129                                                                                 
 lemmas | spatial dealiasing seismic reflection datum main challenge reflection seismology provide spatial sampling require avo... 
-RECORD 1--------------------------------------------------------------------------------------------------------------------------
 id     | 30e2c1e54a88f22513458f53129e04de590ee515                                                                                 
 lemmas | low-pressure spray form aluminum alloy abstract paper develop low-pressure spray form technique describe experimental... 
-RECORD 2--------------------------------------------------------------------------------------------------------------------------
 id     | aae5e57ccb5555bf83e3ba8a78c37d1f5cb434b2                          

In [19]:
cs_lemmas = (S2lemmas.join(papers_s2cs, S2lemmas.id == papers_s2cs.id, "right")
                     .drop(papers_s2cs.id)
                     .withColumn("fieldsOfStudy", lit("computer_science"))
            )
cs_lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 22d91e5c3cc5c2a2d9d3a6371dd3fe45d751a612                                                                                 
 lemmas        |                                                                                                                          
 year          | 2019                                                                                                                     
 fieldsOfStudy | computer_science                                                                                                         
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 2a5d279daf91532621cd36f77047ef52d9d90568                                                                                 
 lemmas        | boolean fu

In [20]:
eco_lemmas = (S2lemmas.join(papers_s2eco, S2lemmas.id == papers_s2eco.id, "right")
                     .drop(papers_s2eco.id)
                     .withColumn("fieldsOfStudy", lit("economics"))
            )
eco_lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 06dfb45d1b3fcc3de52cad5213aad677832eefe3                                                                                 
 lemmas        | first edition nominate book year second edition ask more money spend help world poor people spend use common framewor... 
 year          | 2004                                                                                                                     
 fieldsOfStudy | economics                                                                                                                
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 112654979fd703dd32df9202f3315a32da75569a                                                                                 
 lemmas        | introduce 

In [21]:
socio_lemmas = (S2lemmas.join(papers_s2socio, S2lemmas.id == papers_s2socio.id, "right")
                     .drop(papers_s2socio.id)
                     .withColumn("fieldsOfStudy", lit("sociology"))
            )
socio_lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 0cd5cf3d38a9acf8f4744ad0e5a5194983b61ba7                                                                                 
 lemmas        | political stereotype form obstacle enforcement decision article enquire present african political landscape conducive... 
 year          | 2010                                                                                                                     
 fieldsOfStudy | sociology                                                                                                                
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 36730a87890bb7b4547386c42fc0aa61ee5d9b2a                                                                                 
 lemmas        | fellow inf

In [22]:
phi_lemmas = (S2lemmas.join(papers_s2phi, S2lemmas.id == papers_s2phi.id, "right")
                     .drop(papers_s2phi.id)
                     .withColumn("fieldsOfStudy", lit("philosophy"))
            )
phi_lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 0475198e31fabce94dc5186798c5513d15139d26                                                                                 
 lemmas        | carbon nitrogen sequestration follow forage grassland conversion effect co_2 flux take sample forage grassland alfalf... 
 year          | 2009                                                                                                                     
 fieldsOfStudy | environmental_science                                                                                                    
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 12cd49e0b20aa2bce9c584d2c851dfcb7c940571                                                                                 
 lemmas        | measure ab

In [23]:
ps_lemmas = (S2lemmas.join(papers_s2ps, S2lemmas.id == papers_s2ps.id, "right")
                     .drop(papers_s2ps.id)
                     .withColumn("fieldsOfStudy", lit("political_science"))
            )
ps_lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 0fcfb740cdc7a5fa1daf77279c1ae5175ca4b950                                                                                 
 lemmas        | 20th 21st paper investigate socio-economic role waqf 20th century date endure various political fortune improve socia... 
 year          | 2017                                                                                                                     
 fieldsOfStudy | political_science                                                                                                        
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | null                                                                                                                     
 lemmas        | null      

In [24]:
from functools import reduce # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

lemmas_federated = unionAll(cs_lemmas, eco_lemmas, socio_lemmas, phi_lemmas, ps_lemmas)

In [ ]:
print('Number of papers selected:', lemmas_federated.count())

In [ ]:
#lemmas_federated.show(n=2, truncate=120, vertical=True)

## 3. Save lemmas of five selected categories to local

In [ ]:
lemmas_federated.coalesce(1000).write.parquet(f"file://{path_lemmas_federated}", mode="overwrite")

## 4. Preproc

In [5]:
import argparse
import json
import pathlib
import shutil
from pathlib import Path
import sys

import dask.dataframe as dd
import pyarrow as pa
from dask.diagnostics import ProgressBar
from gensim import corpora
import pyspark.sql.functions as F
from pyspark.ml.feature import (CountVectorizer, StopWordsRemover,
                                Tokenizer)
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("Topicmodeling")\
    .getOrCreate()

In [21]:
class textPreproc(object):
    """
    A simple class to carry out some simple text preprocessing tasks
    that are needed by topic modeling
    - Stopword removal
    - Replace equivalent terms
    - Calculate BoW
    - Generate the files that are needed for training of different
      topic modeling technologies

    It allows to use Gensim or Spark functions
    """

    def __init__(self, stw_files=[], eq_files=[],
                 min_lemas=15, no_below=10, no_above=0.6,
                 keep_n=100000, cntVecModel=None,
                 GensimDict=None, logger=None):
        """
        Initilization Method
        Stopwords and the dictionary of equivalences will be loaded
        during initialization

        Parameters
        ----------
        stw_files: list of str
            List of paths to stopwords files
        eq_files: list of str
            List of paths to equivalent terms files
        min_lemas: int
            Minimum number of lemas for document filtering
        no_below: int
            Minimum number of documents to keep a term in the vocabulary
        no_above: float
            Maximum proportion of documents to keep a term in the vocab
        keep_n: int
            Maximum vocabulary size
        cntVecModel : pyspark.ml.feature.CountVectorizerModel
            CountVectorizer Model to be used for the BOW calculation
        GensimDict : gensim.corpora.Dictionary
            Optimized Gensim Dictionary Object
        logger: Logger object
            To log object activity
        """
        self._stopwords = self._loadSTW(stw_files)
        print(self._stopwords)
        self._equivalents = self._loadEQ(eq_files)
        self._min_lemas = min_lemas
        self._no_below = no_below
        self._no_above = no_above
        self._keep_n = keep_n
        self._cntVecModel = cntVecModel
        self._GensimDict = GensimDict

        if logger:
            self._logger = logger
        else:
            import logging
            logging.basicConfig(level='INFO')
            self._logger = logging.getLogger('textPreproc')

    def _loadSTW(self, stw_files):
        """
        Loads all stopwords from all files provided in the argument

        Parameters
        ----------
        stw_files: list of str
            List of paths to stopwords files

        Returns
        -------
        stopWords: list of str
            List of stopwords
        """

        stopWords = []
        for stwFile in stw_files:
            with Path(stwFile).open('r', encoding='utf8') as fin:
                stopWords += json.load(fin)['wordlist']

        return list(set(stopWords))

    def _loadEQ(self, eq_files):
        """
        Loads all equivalent terms from all files provided in the argument

        Parameters
        ----------
        eq_files: list of str
            List of paths to equivalent terms files

        Returns
        -------
        equivalents: dictionary
            Dictionary of term_to_replace -> new_term
        """

        equivalent = {}

        for eqFile in eq_files:
            with Path(eqFile).open('r', encoding='utf8') as fin:
                newEq = json.load(fin)['wordlist']
            newEq = [x.split(':') for x in newEq]
            newEq = [x for x in newEq if len(x) == 2]
            newEq = dict(newEq)
            equivalent = {**equivalent, **newEq}

        return equivalent

    def preprocBOW(self, trDF, nw=0):
        """
        Preprocesses the documents in the dataframe to carry
        out the following tasks
            - Filter out short documents (below min_lemas)
            - Cleaning of stopwords
            - Equivalent terms application
            - BoW calculation

        Parameters
        ----------
        trDF: Dask or Spark dataframe
            This routine works on the following column "all_lemmas"
            Other columns are left untouched
        nw: Number of workers to use if Dask is selected
            If nw=0 use Dask default value (number of cores)

        Returns
        -------
        trDFnew: A new dataframe with a new colum bow containing the
        bow representation of the documents
        """
        if isinstance(trDF, dd.DataFrame):

            def tkz_clean_str(rawtext):
                """Function to carry out tokenization and cleaning of text

                Parameters
                ----------
                rawtext: str
                    string with the text to lemmatize

                Returns
                -------
                cleantxt: str
                    Cleaned text
                """
                if rawtext == None or rawtext == '':
                    return ''
                else:
                    # lowercase and tokenization (similar to Spark tokenizer)
                    cleantext = rawtext.lower().split()
                    # remove stopwords
                    cleantext = [
                        el for el in cleantext if el not in self._stopwords]
                    # replacement of equivalent words
                    cleantext = [self._equivalents[el] if el in self._equivalents else el
                                 for el in cleantext]
                return cleantext

            # Compute tokens, clean them, and filter out documents
            # with less than minimum number of lemmas
            trDF['final_tokens'] = trDF['all_lemmas'].apply(
                tkz_clean_str, meta=('all_lemmas', 'object'))
            trDF = trDF.loc[trDF.final_tokens.apply(
                len, meta=('final_tokens', 'int64')) >= self._min_lemas]

            # Gensim dictionary creation. It persists the created Dataframe
            # to accelerate dictionary calculation
            # Filtering of words is carried out according to provided values
            self._logger.info('-- -- Gensim Dictionary Generation')

            with ProgressBar():
                DFtokens = trDF[['final_tokens']]
                if nw>0:
                    DFtokens = DFtokens.compute(scheduler='processes', num_workers=nw)
                else:
                    #Use Dask default (i.e., number of available cores)
                    DFtokens = DFtokens.compute(scheduler='processes')
            self._GensimDict = corpora.Dictionary(
                DFtokens['final_tokens'].values.tolist())

            # Remove words that appear in less than no_below documents, or in more than
            # no_above, and keep at most keep_n most frequent terms

            self._logger.info('-- -- Gensim Filter Extremes')

            self._GensimDict.filter_extremes(no_below=self._no_below,
                                             no_above=self._no_above, keep_n=self._keep_n)

            # We skip the calculation of the bow for each document, because Spark LDA will
            # not be used in this case. Note that this is different from what is done for
            # Spark preprocessing
            trDFnew = trDF

        else:
            # Preprocess data using Spark
            # tokenization
            tk = Tokenizer(inputCol="all_lemmas", outputCol="tokens")
            trDF = tk.transform(trDF)

            # Removal of Stopwords - Skip if not stopwords are provided
            # to save computation time
            if len(self._stopwords):
                swr = StopWordsRemover(inputCol="tokens", outputCol="clean_tokens",
                                       stopWords=self._stopwords)
                trDF = swr.transform(trDF)
            else:
                # We need to create a copy of the tokens with the new name
                trDF = trDF.withColumn("clean_tokens", trDF["tokens"])

            # Filter according to number of lemmas in each document
            trDF = trDF.where(F.size(F.col("clean_tokens")) >= self._min_lemas)

            # Equivalences replacement
            if len(self._equivalents):
                df = trDF.select(trDF.id, F.explode(trDF.clean_tokens))
                df = df.na.replace(self._equivalents, 1)
                df = df.groupBy("id").agg(F.collect_list("col"))
                trDF = (trDF.join(df, trDF.id == df.id, "left")
                        .drop(df.id)
                        .withColumnRenamed("collect_list(col)", "final_tokens")
                        )
            else:
                # We need to create a copy of the tokens with the new name
                trDF = trDF.withColumn("final_tokens", trDF["clean_tokens"])

            if not self._cntVecModel:
                cntVec = CountVectorizer(inputCol="final_tokens",
                                         outputCol="bow", minDF=self._no_below,
                                         maxDF=self._no_above, vocabSize=self._keep_n)
                self._cntVecModel = cntVec.fit(trDF)

            trDFnew = (self._cntVecModel.transform(trDF)
                           .drop("tokens", "clean_tokens", "final_tokens")
                       )

        return trDFnew

    def saveCntVecModel(self, dirpath):
        """
        Saves a Count Vectorizer Model to the specified path
        Saves also a text document with the corresponding
        vocabulary

        Parameters
        ----------
        dirpath: pathlib.Path
            The folder where the CountVectorizerModel and the
            text file with the vocabulary will be saved

        Returns
        -------
        status: int
            - 1: If the files were generated sucessfully
            - 0: Error (Count Vectorizer Model does not exist)
        """
        if self._cntVecModel:
            cntVecModel = dirpath.joinpath('CntVecModel')
            if cntVecModel.is_dir():
                shutil.rmtree(cntVecModel)
            self._cntVecModel.save(f"file://{cntVecModel.as_posix()}")
            with dirpath.joinpath('vocabulary.txt').open('w', encoding='utf8') as fout:
                fout.write(
                    '\n'.join([el for el in self._cntVecModel.vocabulary]))
            return 1
        else:
            return 0

    def saveGensimDict(self, dirpath):
        """
        Saves a Gensim Dictionary to the specified path
        Saves also a text document with the corresponding
        vocabulary

        Parameters
        ----------
        dirpath: pathlib.Path
            The folder where the Gensim dictionary and the
            text file with the vocabulary will be saved

        Returns
        -------
        status: int
            - 1: If the files were generated sucessfully
            - 0: Error (Gensim dictionary does not exist)
        """
        if self._GensimDict:
            GensimFile = dirpath.joinpath('dictionary.gensim')
            if GensimFile.is_file():
                GensimFile.unlink()
            self._GensimDict.save_as_text(GensimFile)
            with dirpath.joinpath('vocabulary.txt').open('w', encoding='utf8') as fout:
                fout.write(
                    '\n'.join([self._GensimDict[idx] for idx in range(len(self._GensimDict))]))
            return 1
        else:
            return 0

    def exportTrData(self, trDF, dirpath, tmTrainer, nw=0):
        """
        Exports the training data in the provided dataset to the
        format required by the topic modeling trainer

        Parameters
        ----------
        trDF: Dask or Spark dataframe
            If Spark, the dataframe should contain a column "bow" that will
            be used to calculate the training data
            If Dask, it should contain a column "final_tokens"
        dirpath: pathlib.Path
            The folder where the data will be saved
        tmTrainer: string
            The output format [mallet|sparkLDA|prodLDA|ctm]
        nw: Number of workers to use if Dask is selected
            If nw=0 use Dask default value (number of cores)

        Returns
        -------
        outFile: Path
            A path containing the location of the training data in the indicated format
        """

        self._logger.info(f'-- -- Exporting corpus to {tmTrainer} format')

        if isinstance(trDF, dd.DataFrame):
            # Dask dataframe

            # Remove words not in dictionary, and return a string
            vocabulary = set([self._GensimDict[idx]
                             for idx in range(len(self._GensimDict))])

            def tk_2_text(tokens):
                """Function to filter words not in dictionary, and
                return a string of lemmas 

                Parameters
                ----------
                tokens: list
                    list of "final_tokens"

                Returns
                -------
                lemmasstr: str
                    Clean text including only the lemmas in the dictionary
                """
                #bow = self._GensimDict.doc2bow(tokens)
                # return ''.join([el[1] * (self._GensimDict[el[0]]+ ' ') for el in bow])
                return ' '.join([el for el in tokens if el in vocabulary])

            trDF['cleantext'] = trDF['final_tokens'].apply(
                tk_2_text, meta=('final_tokens', 'str'))

            if tmTrainer == "mallet":

                outFile = dirpath.joinpath('corpus.txt')
                if outFile.is_file():
                    outFile.unlink()

                trDF['2mallet'] = trDF['id'].apply(
                    str, meta=('id', 'str')) + " 0 " + trDF['cleantext']

                with ProgressBar():
                    #trDF = trDF.persist(scheduler='processes')
                    DFmallet = trDF[['2mallet']]
                    if nw>0:
                        DFmallet.to_csv(outFile, index=False, header=False, single_file=True,
                                    compute_kwargs={'scheduler': 'processes', 'num_workers': nw})
                    else:
                        #Use Dask default number of workers (i.e., number of cores)
                        DFmallet.to_csv(outFile, index=False, header=False, single_file=True,
                                    compute_kwargs={'scheduler': 'processes'})

            elif tmTrainer == 'sparkLDA':
                self._logger.error(
                    '-- -- sparkLDA requires preprocessing with spark')
                return

            elif tmTrainer == "prodLDA":

                outFile = dirpath.joinpath('corpus.parquet')
                if outFile.is_file():
                    outFile.unlink()

                with ProgressBar():
                    DFparquet = trDF[['id', 'cleantext']].rename(
                        columns={"cleantext": "bow_text"})
                    if nw>0:
                        DFparquet.to_parquet(outFile, write_index=False, compute_kwargs={
                                         'scheduler': 'processes', 'num_workers': nw})
                    else:
                        #Use Dask default number of workers (i.e., number of cores)
                        DFparquet.to_parquet(outFile, write_index=False, compute_kwargs={
                                         'scheduler': 'processes'})

            elif tmTrainer == "ctm":
                outFile = dirpath.joinpath('corpus.parquet')
                if outFile.is_file():
                    outFile.unlink()

                with ProgressBar():
                    # DFparquet = trDF[['id', 'cleantext', 'all_rawtext']].rename(
                    #    columns={"cleantext": "bow_text"})
                    DFparquet = trDF[['id', 'cleantext', 'embeddings']].rename(
                        columns={"cleantext": "bow_text"})
                    schema = pa.schema([
                        ('id', pa.int64()),
                        ('bow_text', pa.string()),
                        ('embeddings', pa.list_(pa.float64()))
                    ])
                    if nw>0:
                        DFparquet.to_parquet(outFile, write_index=False, schema=schema, compute_kwargs={
                                         'scheduler': 'processes', 'num_workers': nw})
                    else:
                        #Use Dask default number of workers (i.e., number of cores)
                        DFparquet.to_parquet(outFile, write_index=False, schema=schema, compute_kwargs={
                                         'scheduler': 'processes'})

        else:
            # Spark dataframe
            if tmTrainer == "mallet":
                # We need to convert the bow back to text, and save text file
                # in mallet format
                outFile = dirpath.joinpath('corpus.txt')
                vocabulary = self._cntVecModel.vocabulary
                spark.sparkContext.broadcast(vocabulary)

                # User defined function to recover the text corresponding to BOW
                def back2text(bow):
                    text = ""
                    for idx, tf in zip(bow.indices, bow.values):
                        text += int(tf) * (vocabulary[idx] + ' ')
                    return text.strip()
                back2textUDF = F.udf(lambda z: back2text(z))

                malletDF = (trDF.withColumn("bow_text", back2textUDF(F.col("bow")))
                            .withColumn("2mallet", F.concat_ws(" 0 ", "id", "bow_text"))
                            .select("2mallet")
                            )
                # Save as text file
                # Ideally everything should get written to one text file directly from Spark
                # but this is failing repeatedly, so I avoid coalescing in Spark and
                # instead concatenate all files after creation
                tempFolder = dirpath.joinpath('tempFolder')
                #malletDF.coalesce(1).write.format("text").option("header", "false").save(f"file://{tempFolder.as_posix()}")
                malletDF.write.format("text").option("header", "false").save(
                    f"file://{tempFolder.as_posix()}")
                # Concatenate all text files
                with outFile.open("w", encoding="utf8") as fout:
                    for inFile in [f for f in tempFolder.iterdir() if f.name.endswith('.txt')]:
                        fout.write(inFile.open("r").read())
                shutil.rmtree(tempFolder)

            elif tmTrainer == "sparkLDA":
                # Save necessary columns for Spark LDA in parquet file
                outFile = dirpath.joinpath('corpus.parquet')
                trDF.select("id", "source", "bow").write.parquet(
                    f"file://{outFile.as_posix()}", mode="overwrite")
            elif tmTrainer == "prodLDA":
                outFile = dirpath.joinpath('corpus.parquet')
                lemas_df = (trDF.withColumn("bow_text", back2textUDF(
                    F.col("bow"))).select("id", "bow_text"))
                lemas_df.write.parquet(
                    f"file://{outFile.as_posix()}", mode="overwrite")
            elif tmTrainer == "ctm":
                outFile = dirpath.joinpath('corpus.parquet')
                
                vocabulary = self._cntVecModel.vocabulary
                spark.sparkContext.broadcast(vocabulary)

                # User defined function to recover the text corresponding to BOW
                def back2text(bow):
                    text = ""
                    for idx, tf in zip(bow.indices, bow.values):
                        text += int(tf) * (vocabulary[idx] + ' ')
                    return text.strip()
                back2textUDF = F.udf(lambda z: back2text(z))
                
                
                lemas_raw_df = (trDF.withColumn("bow_text", back2textUDF(
                    F.col("bow"))).select("id", "bow_text", "embeddings", "fieldsOfStudy"))
                lemas_raw_df.write.parquet(
                    f"file://{outFile.as_posix()}", mode="overwrite")

In [23]:
train_config = {
    "TrDtSet": "/export/usuarios_ml4ds/lbartolome/gFedNTM/aux/datasets/scholar_dtst.json",
    "Preproc": {
      "min_lemas": 15,
      "no_below": 10,
      "no_above": 0.6,
      "keep_n": 500000,
      "stopwords": [
        "/export/usuarios_ml4ds/lbartolome/gFedNTM/aux/wordlists/S2_stopwords.json"
      ],
      "equivalences": [
        "/export/usuarios_ml4ds/lbartolome/gFedNTM/aux/wordlists/S2_equivalences.json"
      ]
    }
  }

In [25]:
tPreproc = textPreproc(
    stw_files=train_config['Preproc']['stopwords'],
    eq_files=train_config['Preproc']['equivalences'],
    min_lemas=train_config['Preproc']['min_lemas'],
    no_below=train_config['Preproc']['no_below'],
    no_above=train_config['Preproc']['no_above'],
    keep_n=train_config['Preproc']['keep_n'])

['eeective', '1a', 'eld', 'eectiveness', 'afford', 'ambitious', 'ea', 'duce', 'increased', 'definition', 'community', 'dence', 'arrange', 'arti', 'advance', 'chap', 'eeort', 'dia', 'elaborate', 'da', 'confi', 'cla', 'diff', 'description', 'constituent', 'er', 'det', 'compatibility', 'anal', 'designer', 'purposethe', 'work', 'article', 'confine', 'appraisal', 'design', 'dv', 'world', 'existent', 'exemplify', 'doc', 'av', 'beneficial', 'aspect', 'new', 'ba', 'upcoming', 'contribution', 'amendment', 'ence', 'ecp', 'choose', 'information', 'conclusion', 'system', 'behavior', 'emphasise', 'adequacy', 'a_number', 'inner', 'ective', 'challenging', 'def', 'separated', 'consequent', 'download', 'bl', 'cent', 'eration', 'appendix', 'cate', 'entail', 'bd', 'di', 'closer', 'categorise', 'deem', 'cre', 'ci', 'decade', 'numerous', 'demon', 'characteristic', 'coe', 'basics', 'deter', 'device', 'defi', '3rd', 'difficulty', 'dr', 'attempt', 'behaviour', 'cha', 'ect', 'dat', 'bm', 'dierence', 'comprehen

In [26]:
# Create a Dataframe with all training data
trDtFile = Path(train_config['TrDtSet'])
with trDtFile.open() as fin:
    trDtSet = json.load(fin)
trDtSet

{'name': 'Scholar',
 'description': 'Topic modeling training dataset of a five subsets of Scholar',
 'valid_for': 'TM',
 'visibility': 'private',
 'Dtsets': [{'parquet': '/export/usuarios_ml4ds/lbartolome/data/scholar_federated_small',
   'source': 'Scholar',
   'idfld': 'id',
   'lemmasfld': ['lemmas'],
   'filter': ''}],
 'creation_date': '2022-09-17 17:31:13.601336'}

In [27]:
for idx, DtSet in enumerate(trDtSet['Dtsets']):
    df = spark.read.parquet(f"file://{DtSet['parquet']}")
    if len(DtSet['filter']):
        pass
    df = (
        df.withColumn("all_lemmas", F.concat_ws(
            ' ', *DtSet['lemmasfld']))
            .withColumn("source", F.lit(DtSet["source"]))
            .select("id", "source", "all_lemmas")
    )
    if idx == 0:
        trDF = df
    else:
        trDF = trDF.union(df).distinct()

In [28]:
trDF = tPreproc.preprocBOW(trDF)

/opt/spark-3.1.1-bin-2.8.3/python/pyspark/sql/dataframe.py:2213: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [29]:
tPreproc.saveCntVecModel(Path("/export/usuarios_ml4ds/lbartolome/gFedNTM/aux"))

22/10/22 18:33:52 WARN scheduler.TaskSetManager: Stage 54 contains a task of very large size (2073 KiB). The maximum recommended task size is 1000 KiB.


1

In [30]:
#for idx, DtSet in enumerate(trDtSet['Dtsets']):
df = spark.read.parquet(f"file://{path_embeddings}")
df = df.select("id", "embeddings", "fieldsOfStudy")
eDF = df

In [31]:
trDF = (trDF.join(eDF, trDF.id == eDF.id, "left")
                    .drop(eDF.id))

In [32]:
#trDF.show(n=2, truncate=120, vertical=True)

In [33]:
papers_s2cs = trDF.filter(F.array_contains("fieldsOfStudy", 'Computer Science')).withColumn("fieldsOfStudy", lit("computer_science"))
count_s2cs = papers_s2cs.count()
print('Number of papers in Computer Science:', count_s2cs)

papers_s2eco = trDF.filter(F.array_contains("fieldsOfStudy", 'Economics')).withColumn("fieldsOfStudy", lit("economics"))
count_s2eco = papers_s2eco.count()
print('Number of papers in Economics:', count_s2eco)

papers_s2socio = trDF.filter(F.array_contains("fieldsOfStudy", 'Sociology')).withColumn("fieldsOfStudy", lit("sociology"))
count_s2socio =  papers_s2socio.count()
print('Number of papers in Sociology:', count_s2socio)

papers_s2phi = trDF.filter(F.array_contains("fieldsOfStudy", 'Philosophy')).withColumn("fieldsOfStudy", lit("philosophy"))
count_s2phi = papers_s2phi.count()
print('Number of papers in Environmental Science:', count_s2phi)

papers_s2ps = trDF.filter(F.array_contains("fieldsOfStudy", 'Political Science')).withColumn("fieldsOfStudy", lit("political_science"))
count_s2ps = papers_s2ps.count()
print('Number of papers in Political Science:', count_s2ps)

Number of papers in Computer Science: 732039


Number of papers in Economics: 616261


Number of papers in Sociology: 440139


Number of papers in Environmental Science: 132545


Number of papers in Political Science: 304195


In [34]:
from functools import reduce # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

trDF = unionAll(papers_s2cs, papers_s2eco, papers_s2socio, papers_s2phi, papers_s2ps)

In [35]:
trDataFile = tPreproc.exportTrData(trDF=trDF,
                                   dirpath=Path("/export/usuarios_ml4ds/lbartolome/gFedNTM/aux"),
                                   tmTrainer='ctm')

INFO:textPreproc:-- -- Exporting corpus to ctm format
22/10/22 23:11:24 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1662.6 KiB


In [36]:
lemmas = spark.read.parquet(f"file://{path_output}")
lemmas.show(n=2, truncate=120, vertical=True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------
 id            | 0008e1e1fafabd8e5c7418d5cbc74ace257bde81                                                                                 
 bow_text      | order first field detection detection second medium medium medium medium target target transmission associate one one... 
 embeddings    | [0.03201526030898094, 0.017998920753598213, 0.04557495936751366, -0.06096094474196434, 0.006882247515022755, -0.04495... 
 fieldsOfStudy | computer_science                                                                                                         
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------
 id            | 000c9d65ac49b9a32235ab8b8038720f80f58112                                                                                 
 bow_text      | present pr

In [37]:
print('Number of papers:', lemmas.count())

Number of papers: 2225179


2022-10-23 00:53:56,044:33340(0x7f9a7d7dc6c0):ZOO_WARN@zookeeper_interest@1597: Exceeded deadline by 41ms
